#### Package Import

In [1]:
import myqengine as qe
import mypaengine as pa
import pandas as pd
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Screening for IDs & Characteristics using Quant Engine

#### 1. Get All Mutual Funds that Hold Nvidia

##### a. Query data using FQL

In [2]:
#set universe to NVDA-US
univ = qe.id_universe(ids = ['NVDA-US'],
                    universe_type ='Equity')

#set to latest month end
time_series = qe.time_series(start_date='0M',
                            end_date = '0M')

#Define FQL Formulas
formulas = ['OS_TOP_HLDR_ID(ALL,#DATE,,M,,M)','OS_TOP_HLDR_NAME(ALL,#DATE,,M,,M,,"EN")','OS_TOP_HLDR_MV(ALL,#DATE,,M,,M,SEC,USD)']

#Calculate
q_req = qe.calculate(universe=univ, dates = time_series, formulas=formulas,is_array_return_type=True,source= 'FqlExpression')


##### b. Data prep

In [3]:
df_temp = q_req.data.copy()

#clean up table
df_temp.rename(columns ={'OS_TOP_HLDR_ID(ALL,#DATE,,M,,M)':'hldr_id',
                    'OS_TOP_HLDR_NAME(ALL,#DATE,,M,,M,,"EN")':'hldr_name',
                    'OS_TOP_HLDR_MV(ALL,#DATE,,M,,M,SEC,USD)':'position_mv',
                    },inplace=True)
                    

#Expand arrays and rejoin
df = pd.concat([df_temp['hldr_id'].explode() ,df_temp['hldr_name'].explode(),df_temp['position_mv'].explode()],axis=1)
df.set_index('hldr_id',inplace=True)
df.head()

,hldr_name,position_mv
hldr_id,,
M4004630,Vanguard Total Stock Market ETF,34196140611.130001
M4004543,Vanguard 500 Index Fund,27676648381.68
M4001597,Fidelity 500 Index Fund,13049993344.01
M4006564,SPDR S&P 500 ETF Trust,13019070428.26
M4032427,Government Pension Fund - Global (The),12366990597.76


#### 2. Get exchange ticker and other additional data items for holder IDs returned

##### a. Query data using FQL

In [4]:
#Set universe to hldr_ids from previous step
fund_univ = qe.id_universe(ids = df.index.to_list(),universe_type ='Equity')

#Set Formulas using Holder Formulas
formulas = ['OS_HLDR_MSTYLE','OS_HLDR_MF_FAMILY','STRING(FIRST_ITEM_AV(OS_FUND_TICKER))','FFD_AUM(#DATE,,M,USD)']

#Calculate
q_req = qe.calculate(universe=fund_univ, dates = time_series, formulas=formulas,source= 'FqlExpression')

##### b. Data prep

In [5]:
#Clean up table
dff = q_req.data.copy()

dff.rename(columns= {"UNIVERSE":"hldr_id",
                    'OS_HLDR_MSTYLE':'style',
                    'OS_HLDR_MF_FAMILY':'fund_family',
                    'STRING(FIRST_ITEM_AV(OS_FUND_TICKER))':'ticker',
                    'FFD_AUM(#DATE,,M,USD)':'aum'
                    },inplace=True)
dff.set_index(['hldr_id'],inplace=True)

#join datasets
df = df.join(dff,how='left')
#calculate 
df['weight_in_nvda'] =df['position_mv'] /df['aum']

#Replace @NA Groupings with Other
df['fund_family']=df['fund_family'].replace('@NA','Other')
df['style']=df['style'].replace('@NA','Other')
df = df[df['ticker']!='@NA']
#Clean up table
df[df['ticker']!='']
df = df.replace('',np.nan)
df = df.dropna()

#preview
df.head()

,hldr_name,position_mv,DATE,style,fund_family,ticker,aum,weight_in_nvda
hldr_id,,,,,,,,
M4004630,Vanguard Total Stock Market ETF,3.419614e+10,20230731,Generalist,Vanguard Funds,VTSMX,1.393315e+12,0.024543
M4004543,Vanguard 500 Index Fund,2.767665e+10,20230731,Generalist,Vanguard Funds,VFINX,9.205360e+11,0.030066
M4001597,Fidelity 500 Index Fund,1.304999e+10,20230731,Index,Other,FXAIX,4.327432e+11,0.030156
M4006564,SPDR S&P 500 ETF Trust,1.301907e+10,20230731,Generalist,SPDR Funds,SPY,4.320715e+11,0.030132
M4006774,iShares Core S&P 500 ETF,1.071577e+10,20230731,Generalist,iShares Funds,IVV,3.559845e+11,0.030102


#### 3. Get additional classificaiton and trailing performance data
##### a. Query dataset using Screening

In [6]:
#Define Universe Using market ticker list from previous screen
fund_univ = qe.id_universe(ids = df['ticker'].to_list(),universe_type ='Equity')

#Define FactSet Mutual Funds and Other FactSet Screening Formulas
formulas = ['FFD_CLASS_CAT(TEXT)','FFD_CLASS_FOCUS(TEXT)','FFD_CLASS_NICHE(TEXT)','P_PRICE_RETURNS(1,0,-1/0/0,USD)','P_PRICE_RETURNS(1,12/31/0,0,USD)','P_PRICE_RETURNS(1,0/0/-1,0,USD)','LSS_TICKER','FFD_BMK_ID']
q_req = qe.calculate(universe=fund_univ, dates = time_series, formulas=formulas,source= 'ScreeningExpression')
                    


##### b. Data prep

In [7]:
#Rename Columns and cleanup
dff = q_req.data.copy()
dff.rename(columns= {"UNIVERSE":"ticker",
                    'FFD_CLASS_CAT(TEXT)':'category',
                    'FFD_CLASS_FOCUS(TEXT)':'focus',
                    'FFD_CLASS_NICHE(TEXT)':'niche',
                    'P_PRICE_RETURNS(1,0,-1/0/0,USD)':'tret_1m',
                    'P_PRICE_RETURNS(1,12/31/0,0,USD)':'tret_ytd',
                    'P_PRICE_RETURNS(1,0/0/-1,0,USD)':'tret_1y',
                    'LSS_TICKER':'lionshares_ticker',
                    'FFD_BMK_ID':'bench_id',
                    },inplace=True)
dff.set_index('ticker',inplace=True)
#Join to main table
dff= dff.drop(columns = 'DATE')
df = df.join(dff,how='left',on='ticker')


df = df.dropna()
df['category']=df['category'].replace('None','Other')
df['focus']=df['focus'].replace('None','Other')
df['niche']=df['niche'].replace('None','Other')
df.head()

,hldr_name,position_mv,DATE,style,fund_family,ticker,aum,weight_in_nvda,category,focus,niche,tret_1m,tret_ytd,tret_1y,lionshares_ticker,bench_id
hldr_id,,,,,,,,,,,,,,,,
M4004630,Vanguard Total Stock Market ETF,3.419614e+10,20230731,Generalist,Vanguard Funds,VTSMX,1.393315e+12,0.024543,Size and Style,Total Market,Broad-based,3.56942,-4.30127,12.2318,VTSMX-US,CRSPTM
M4001597,Fidelity 500 Index Fund,1.304999e+10,20230731,Index,Other,FXAIX,4.327432e+11,0.030156,Size and Style,Large Cap,Broad-based,3.19463,-3.84664,12.7262,FXAIX-US,SP50
M4006564,SPDR S&P 500 ETF Trust,1.301907e+10,20230731,Generalist,SPDR Funds,SPY,4.320715e+11,0.030132,Size and Style,Large Cap,Broad-based,3.27333,-3.46687,12.6999,SPY-US,SP50
M4006774,iShares Core S&P 500 ETF,1.071577e+10,20230731,Generalist,iShares Funds,IVV,3.559845e+11,0.030102,Size and Style,Large Cap,Broad-based,3.24651,-3.44865,12.6765,IVV-US,SP50
M4017554,Invesco QQQ Trust,9.273455e+09,20230731,Generalist,PowerShares Funds,QQQ,2.134556e+11,0.043444,Size and Style,Large Cap,Broad-based,3.86010,-4.62885,22.3072,QQQ-US,12490H10


### Leveraging Bottoms-Up Analysing using the PA Engine 
#### 1. Calculate Relative Contribution Over the Trailing 1Yr As of Latest Month End

##### a. Define PA Document Inputs

In [8]:
#Set PA Inputs
pa_doc = pa.DocumentTemplate(pa_document_name='PERSONAL:API_REPORTS_SINGLE',
                            pa_component_category='main / Contribution No Group',
                            pa_component_name='Contribution',
)

##### b. Calculate Contribution Report over the trailing year as of month end using primary benchmark

In [9]:
#Sort by total fund AUM
df = df.sort_values('aum',ascending= False)

#filter on desired benchmarks
benchs_of_interest = ['SP50','R.1000','R.1000G','R1000V','R.2000','R.2000G','R2000V','R.3000','R.3000G','R3000V','CRSPTM','OEX','892400','R.MID','R.MIDG','R.MIDV','98900023','CRSPMI','990100','984000','899800']
dfd = df[df['bench_id'].isin(benchs_of_interest)]


#Add new column in PA compatible format
dfd['pa_ticker'] = 'LION:'+dfd['lionshares_ticker']
dfd['pa_bench'] = 'BENCH:'+dfd['bench_id']

#Filter to the top 250 by AUM 
df_f = dfd.head(250)

pa_doc.run_calc(portfolios = df_f['pa_ticker'].to_list(),benchmarks= df_f['pa_bench'].to_list(),start_date = '0M-1AY',
                            end_date = '0M')

#append all outputs to mutual dataframe                            
df_full= pd.DataFrame()
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_bench'] = x
    df_full = pd.concat([df_full,temp])
#Add Column with lionshares formatted ticker
pattern = r'LION:(.*?)BENCH'
df_full['lionshares_ticker'] = df_full['port_bench'].str.extract(pattern, expand=False)

df_full.head()

,Symbol,total0,Beginning+Price,Ending+Price,Price+Change+(%),Ticker-Region,Port.+Beginning+Weight,Port.+Ending+Weight,Total+Return,Contribution+To Return,Port.+Ending+Market+Value,Bench.+Total+Return,Bench.+Total+Return+( Local ),Bench.+Contribution+To Return,Bench.+Contribution+to Return+( Local ),Bench.+Ending+Weight,Bench.+Beginning+Weight,port_bench,lionshares_ticker
0,,Total,,,10.721968,,100.0,100.0,12.574793,12.574793,1395861073449.860107,12.097229,12.097229,12.097229,12.097229,100.0,100.0,LION:VTSMX-USBENCH:CRSPTM,VTSMX-US
1,67066G104,NVIDIA Corporation,181.630005,467.290009,157.275794,NVDA-US,1.057388,2.45247,157.483359,1.793417,34233073343.57,157.420963,157.420963,1.935867,1.935867,2.656486,1.154668,LION:VTSMX-USBENCH:CRSPTM,VTSMX-US
2,037833100,Apple Inc.,162.509995,196.449997,20.884884,AAPL-US,6.11534,6.557492,21.607789,1.336129,91533484739.0,21.390545,21.390545,1.428384,1.428384,7.102629,6.677755,LION:VTSMX-USBENCH:CRSPTM,VTSMX-US
3,594918104,Microsoft Corporation,280.73999,335.920013,19.655203,MSFT-US,5.138628,5.579709,20.811843,1.10571,77884992609.839996,20.463931,20.463931,1.166614,1.166614,6.043603,5.611242,LION:VTSMX-USBENCH:CRSPTM,VTSMX-US
4,30303M102,Meta Platforms Inc. Class A,159.100006,318.600006,100.251419,META-US,0.893091,1.57439,100.251419,0.888508,21976302547.799999,100.251391,100.251391,0.957075,0.957075,1.705342,0.975173,LION:VTSMX-USBENCH:CRSPTM,VTSMX-US


#### 4. Join Final Dataset

In [10]:
#join to mutual dataframe on lionshares ticker
df_final = dfd.set_index('lionshares_ticker').join(df_full.set_index('lionshares_ticker'),how='left')
df_final

,hldr_name,position_mv,DATE,style,fund_family,ticker,aum,weight_in_nvda,category,focus,...,Total+Return,Contribution+To Return,Port.+Ending+Market+Value,Bench.+Total+Return,Bench.+Total+Return+( Local ),Bench.+Contribution+To Return,Bench.+Contribution+to Return+( Local ),Bench.+Ending+Weight,Bench.+Beginning+Weight,port_bench
lionshares_ticker,,,,,,,,,,,,,,,,,,,,,
ACFOX-US,American Century Focused Dynamic Growth Fund,1.413169e+08,20230731,Generalist,American Century Funds,ACFOX,1.805597e+09,0.078266,Size and Style,Large Cap,...,13.981717,13.981717,1805173306.114,17.102878,17.102878,17.102878,17.102878,100.0,100.0,LION:ACFOX-USBENCH:R.1000G
ACFOX-US,American Century Focused Dynamic Growth Fund,1.413169e+08,20230731,Generalist,American Century Funds,ACFOX,1.805597e+09,0.078266,Size and Style,Large Cap,...,157.483359,6.23363,141316907.22,157.420963,157.420963,4.207451,4.207451,5.288649,2.368783,LION:ACFOX-USBENCH:R.1000G
ACFOX-US,American Century Focused Dynamic Growth Fund,1.413169e+08,20230731,Generalist,American Century Funds,ACFOX,1.805597e+09,0.078266,Size and Style,Large Cap,...,40.939318,1.470025,61742078.8,40.939356,40.939356,0.209669,0.209669,0.53962,0.410782,LION:ACFOX-USBENCH:R.1000G
ACFOX-US,American Century Focused Dynamic Growth Fund,1.413169e+08,20230731,Generalist,American Century Funds,ACFOX,1.805597e+09,0.078266,Size and Style,Large Cap,...,27.543855,1.232681,64513525.96,27.543934,27.543934,0.019312,0.019312,0.024891,0.047667,LION:ACFOX-USBENCH:R.1000G
ACFOX-US,American Century Focused Dynamic Growth Fund,1.413169e+08,20230731,Generalist,American Century Funds,ACFOX,1.805597e+09,0.078266,Size and Style,Large Cap,...,14.120382,1.162294,156635861.4,14.120411,14.120411,0.291224,0.291224,3.26513,3.018963,LION:ACFOX-USBENCH:R.1000G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VWUSX-US,Vanguard US Growth Fund,2.712100e+09,20230731,Growth,Vanguard Funds,VWUSX,4.031915e+10,0.067266,Size and Style,Large Cap,...,-45.458618,-0.255286,,,,,,,,LION:VWUSX-USBENCH:R.1000G
VWUSX-US,Vanguard US Growth Fund,2.712100e+09,20230731,Growth,Vanguard Funds,VWUSX,4.031915e+10,0.067266,Size and Style,Large Cap,...,-28.295443,-0.378855,348975559.56,-25.955271,-25.955271,-0.004688,-0.004688,,0.018315,LION:VWUSX-USBENCH:R.1000G
VWUSX-US,Vanguard US Growth Fund,2.712100e+09,20230731,Growth,Vanguard Funds,VWUSX,4.031915e+10,0.067266,Size and Style,Large Cap,...,-0.941101,-0.607248,2561333798.16,-0.94102,-0.94102,-0.620812,-0.620812,5.736588,6.49568,LION:VWUSX-USBENCH:R.1000G


#### 5. Archive Data

In [5]:
import pandas as pd
df_full.to_csv('nvidia_pa_contribution_full_raw_dataset_20230828.csv')
df_final.to_csv('nvidia_holder_analysis_full_dataset_20230828.csv')